In [ ]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from os.path import exists
#import json
#import scipy.stats as stats
#import math
from sklearn.model_selection import train_test_split
#import seaborn as sb
#from sklearn.ensemble import RandomForestClassifier as rf
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#import sklearn.linear_model as lm
#from sklearn.tree import DecisionTreeClassifier as tree
#from sklearn.neighbors import KNeighborsClassifier as knn
#from xgboost import XGBClassifier as xgb
#from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB as gnb
#from sklearn.ensemble import VotingClassifier
#import sklearn.model_selection as ms
#import sklearn.metrics as sm
#import joblib
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
%autosave 5

In [ ]:
def encodeDF(categoricalDF,baseName):
    ohe = ce.OneHotEncoder(handle_unknown='ignore',return_df=True,use_cat_names=True)
    le = ce.OrdinalEncoder(return_df=True)
    oheDF = ohe.fit_transform(categoricalDF)
    oheColumns = list(oheDF.columns)
    pickle.dump(oheColumns, open(f"../Interim/{baseName}OheColumns.pkl", 'wb'))
    leDF = le.fit_transform(categoricalDF)
    pickle.dump(ohe, open(f"../Interim/{baseName}OneHotEncoder.pkl", 'wb'))
    pickle.dump(le, open(f"../Interim/{baseName}LabelEncoder.pkl", 'wb'))
    return pd.concatenate([oheDF,leDF],axis=1)

In [ ]:
def scaleDF(df,baseName):
    scaler = StandardScaler()
    numericalCols = list(df.columns)
    df[numericalCols] = scaler.fit_transform(df[numericalCols])
    pickle.dump(scaler, open(f"../Interim/{baseName}Scaler.pkl", 'wb'))

In [ ]:
def separateDFBySubtype(df):
    numericalCols = []
    categoricalCols = []
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):
            numericalCols.append(str(col))
        else:
            categoricalCols.append(str(col))
    numericalDF = df[numericalCols]
    categoricalDF = df[categoricalCols]
    pickle.dump(numericalCols, open(f"../Interim/{baseName}NumericalCols.pkl", 'wb'))
    pickle.dump(categoricalCols, open(f"../Interim/{baseName}CategoricalCols.pkl", 'wb'))
    return numericalDF,categoricalDF

In [ ]:
def processTrainData(trainFileName):
    unbalancedDF = pd.read_csv(f"../Data/Interim/{trainFileName}.csv")
    for balanceType in ["Under","Over"]:
        combinedName = trainFileName + balanceType
        y = df[["Y"]]
        df = df.drop("Y",axis = 1)
        numericalDF,categoricalDF = separateDFBySubtype(df)
        scaledDF = scaleDF(numericalDF,combinedName)
        encodedDF = encodeDF(categoricalDF,combinedName)
        finalDF = pd.concatenate([scaledDF,encodedDF,y],axis=1)
        finalDF.to_csv(f"../Data/Processed/{combinedName}Train.csv",index=False)

In [ ]:
def splitData(baseName):
    df = pd.read_csv("../Data/Raw/term-deposit-marketing-2020.csv")
    y = df[["Y"]]
    X = df.drop("Y",axis=1)
    XTrain,XTest,yTrain,yTest = train_test_split(X, y, test_size=0.2,random_state=51)
    TermDepositTrain = XTrain.copy()
    TermDepositTrain['Y'] = yTrain
    TermDepositTrain.to_csv(f'../Data/Interim/{baseName}Train.csv',index=False)
    TermDepositTest = XTest.copy()
    TermDepositTest['Y'] = yTest
    TermDepositTest.to_csv(f'../Data/Interim/{baseName}Test.csv',index=False)

In [ ]:
def main():
    baseName = "TermDeposit"
    splitData(baseName)
    processTrainData(baseName)

In [ ]:
main()